In [27]:
try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    # !pip install -q -U tfx==0.15.0rc0
    print("You can safely ignore the package incompatibility errors.")
except Exception:
    pass


import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"
print(tf.__version__)

import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K
from tensorflow.keras import datasets
from tqdm import tqdm_notebook
from sklearn import metrics as skm

tf.random.set_seed(1228)

You can safely ignore the package incompatibility errors.
2.1.0


In [2]:
(x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [0]:
# create numpy to tensor dataset 
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_data = tf.data.Dataset.from_tensor_slices((x_test, y_test))

# shuffle the data
train_data = train_data.shuffle(1024)


# create preprocess function 
@tf.function
def preprocess_train(x, y):
  x = tf.cast(x, tf.float32)
  x = tf.expand_dims(x, 2)
  x = x / 255.
  # x = tf.image.random_flip_left_right(x, seed=1228)
  # x = tf.image.random_flip_up_down(x, seed=1228)
  # x = tf.image.rot90(x)
  return x, tf.cast(y, tf.float32)

@tf.function
def preprocess_test(x, y):
  x = tf.cast(x, tf.float32)
  x = tf.expand_dims(x, 2)
  x = x / 255.
  return x, tf.cast(y, tf.float32)

In [0]:
train_data = train_data.map(preprocess_train)
test_data = test_data.map(preprocess_test)

In [0]:
train_data = train_data.batch(512).prefetch(128)
test_data = test_data.batch(512).prefetch(128)

In [0]:
train_data

<PrefetchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.float32)>

In [0]:
# counter = 0
# for i in test_data:
#   print(i)
#   print(i[0].shape)
#   print(i[1].shape)
#   plt.imshow(i[0][0].numpy().reshape(28,28), cmap='gray')
#   counter += 1
#   if counter == 1:
#     break

## Base model CNN

In [0]:
# model archtecture 
input_ = tf.keras.layers.Input(shape=(28,28,1))
x = tf.keras.layers.Conv2D(128,
                           (3,3),
                           padding='same',
                           activation='relu',
                           kernel_initializer='he_normal',
                           kernel_regularizer=tf.keras.regularizers.l2(l=0.02),
                           kernel_constraint=tf.keras.constraints.MinMaxNorm())(input_)
x = tf.keras.layers.LayerNormalization()(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.Dropout(0.4, trainable=True)(x)
for i in range(10):
  x = tf.keras.layers.Conv2D(128,
                             (3,3),
                             padding='same',
                             activation='relu',
                             kernel_initializer='he_normal',
                             kernel_regularizer=tf.keras.regularizers.l2(l=0.02),
                             kernel_constraint=tf.keras.constraints.MinMaxNorm())(x)
  x = tf.keras.layers.LayerNormalization()(x)
  x = tf.keras.layers.BatchNormalization()(x)
  if i == 7 :
    x = tf.keras.layers.MaxPool2D()(x)
  x = tf.keras.layers.Dropout(0.4, trainable=True)(x)
x = tf.keras.layers.GlobalAvgPool2D()(x)
x = tf.keras.layers.LayerNormalization()(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.4, trainable=True)(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.LayerNormalization()(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.4, trainable=True)(x)
out_ = tf.keras.layers.Dense(10, activation='softmax')(x)

# define model
model = tf.keras.models.Model(input_, out_)
model.summary()

# Training

# Define loss 
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()

# Define optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.008)

# Define batch metcis summary units
train_loss = tf.keras.metrics.Mean(name='train_loss')
test_loss = tf.keras.metrics.Mean(name='test_loss')
train_acc = tf.keras.metrics.SparseCategoricalAccuracy(name='train_acc')
test_acc = tf.keras.metrics.SparseCategoricalAccuracy(name='test_acc')

# Define train function
# Calculate Gradients, and apply to optimizer
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    pred = model(images, training=True)
    loss = loss_fn(labels, pred)
  g = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(g, model.trainable_variables))
  train_loss(loss)
  train_acc(labels, pred)

# Define test functionm
@tf.function
def test_step(images, labels):
  pred = model(images, training=False)
  loss = loss_fn(labels, pred)
  test_loss(loss)
  test_acc(labels, pred)

# Train
EPOCHS = 10

for epoch in range(EPOCHS):
  train_loss.reset_states()
  test_loss.reset_states()

  train_acc.reset_states()
  test_acc.reset_states()

  for img, label in train_data:
    train_step(img, label)
  
  for img, label in test_data:
    test_step(img, label)

  template = 'Epoch:{} \t Loss:{} \t  Acc:{} \t Val_loss:{} \t Val_acc:{}'
  print(template.format(epoch, train_loss.result(), train_acc.result()*100, test_loss.result(), test_acc.result()*100))

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 28, 28, 128)       1280      
_________________________________________________________________
layer_normalization_16 (Laye (None, 28, 28, 128)       256       
_________________________________________________________________
batch_normalization_16 (Batc (None, 28, 28, 128)       512       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 128)       0         
_________________________________________________________________
dropout_26 (Dropout)         (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 14, 14, 128)       1475

## Define with keras eager execution

In [0]:
# model archtecture 
input_ = tf.keras.layers.Input(shape=(28,28,1))
x = tf.keras.layers.Conv2D(128,
                           (3,3),
                           padding='same',
                           activation='relu',
                          #  kernel_initializer='he_normal',
                           kernel_regularizer=tf.keras.regularizers.l2(l=0.02),
                           kernel_constraint=tf.keras.constraints.MinMaxNorm())(input_)
x = tf.keras.layers.LayerNormalization()(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.Dropout(0.4, trainable=True)(x)
for i in range(10):
  x = tf.keras.layers.Conv2D(128,
                             (3,3),
                             padding='same',
                             activation='relu',
                            #  kernel_initializer='he_normal',
                             kernel_regularizer=tf.keras.regularizers.l2(l=0.02),
                             kernel_constraint=tf.keras.constraints.MinMaxNorm())(x)
  x = tf.keras.layers.LayerNormalization()(x)
  x = tf.keras.layers.BatchNormalization()(x)
  if i == 7 :
    x = tf.keras.layers.MaxPool2D()(x)
  x = tf.keras.layers.Dropout(0.4, trainable=True)(x)
x = tf.keras.layers.GlobalAvgPool2D()(x)
x = tf.keras.layers.LayerNormalization()(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.4, trainable=True)(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.LayerNormalization()(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.4, trainable=True)(x)
out_ = tf.keras.layers.Dense(10, activation='softmax')(x)

# define model
model = tf.keras.models.Model(input_, out_)
model.summary()

optimizer = tf.keras.optimizers.Adam(learning_rate=0.008)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['acc'])
hist = model.fit(train_data, epochs=10, validation_data=test_data)

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_54 (Conv2D)           (None, 28, 28, 128)       1280      
_________________________________________________________________
layer_normalization_42 (Laye (None, 28, 28, 128)       256       
_________________________________________________________________
batch_normalization_42 (Batc (None, 28, 28, 128)       512       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 14, 14, 128)       0         
_________________________________________________________________
dropout_52 (Dropout)         (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_55 (Conv2D)           (None, 14, 14, 128)       1475

# Create Highway networks
### 1) create as plane
### 2) create as function api
### 3) create as sub-class api

In [0]:
tf.keras.backend.clear_session()

# model archtecture 
input_ = tf.keras.layers.Input(shape=(28,28,1))
x = tf.keras.layers.Conv2D(128,
                           (3,3),
                           padding='same',
                           activation='relu',
                          #  kernel_initializer='he_normal',
                           kernel_regularizer=tf.keras.regularizers.l2(l=0.02),
                           kernel_constraint=tf.keras.constraints.MinMaxNorm())(input_)
x = tf.keras.layers.LayerNormalization()(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.Dropout(0.4, trainable=True)(x)
for i in range(10):
  # highway block
  h = tf.keras.layers.Conv2D(128,
                             (3,3),
                             padding='same',
                             activation='relu',
                            #  kernel_initializer='he_normal',
                             kernel_regularizer=tf.keras.regularizers.l2(l=0.02),
                             kernel_constraint=tf.keras.constraints.MinMaxNorm())(x)
  c = tf.keras.layers.Conv2D(128, 
                             (3,3), 
                             padding='same',
                             activation='sigmoid',
                             kernel_regularizer=tf.keras.regularizers.l2(l=0.02),
                             kernel_constraint=tf.keras.constraints.MinMaxNorm(),
                             bias_initializer = tf.keras.initializers.Constant(-3.))(x)
  hc = tf.keras.layers.Multiply()([h,c])
  ad = tf.keras.layers.Lambda(lambda x: 1.0 - x)(c)

  xc = tf.keras.layers.Multiply()([x, ad])
  x = tf.keras.layers.Add()([hc, xc])  
  if i == 7 :
    x = tf.keras.layers.MaxPool2D()(x)
  x = tf.keras.layers.Dropout(0.4, trainable=True)(x)
x = tf.keras.layers.GlobalAvgPool2D()(x)
x = tf.keras.layers.LayerNormalization()(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.4, trainable=True)(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.LayerNormalization()(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.4, trainable=True)(x)
out_ = tf.keras.layers.Dense(10, activation='softmax')(x)

# define model
model = tf.keras.models.Model(input_, out_)
model.summary()

# plot model
tf.keras.utils.plot_model(model, show_shapes=True, dpi=80)

# define loss
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Set summary metrics
train_loss = tf.keras.metrics.Mean()
train_acc = tf.keras.metrics.SparseCategoricalAccuracy()

test_loss = tf.keras.metrics.Mean()
test_acc = tf.keras.metrics.SparseCategoricalAccuracy()


@tf.function
def train_step(img, label):
  with tf.GradientTape() as tape:
    pred = model(img, training=True)
    loss = loss_fn(label, pred)
  grad = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grad, model.trainable_variables))
  train_loss(loss)
  train_acc(label, pred)

@tf.function
def test_step(img, label):
  pred = model(img, training=False)
  loss = loss_fn(label, pred)
  test_loss(loss)
  test_acc(label, pred)


# Train Loop
n_epochs = 50
for epoch in range(n_epochs):
  train_loss.reset_states()
  train_acc.reset_states()
  test_loss.reset_states()
  test_acc.reset_states()

  for img, lab in tqdm_notebook(train_data):
    train_step(img, lab)

  for img, lab in test_data:
    test_step(img, lab)

  template = 'Epoch:{} \t Loss:{} \t  Acc:{} \t Val_loss:{} \t Val_acc:{}'
  print(template.format(epoch, train_loss.result(), train_acc.result()*100, test_loss.result(), test_acc.result()*100))

  

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 28, 28, 128)  1280        input_1[0][0]                    
__________________________________________________________________________________________________
layer_normalization (LayerNorma (None, 28, 28, 128)  256         conv2d[0][0]                     
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 28, 28, 128)  512         layer_normalization[0][0]        
______________________________________________________________________________________________

Epoch:0 	 Loss:2.5990819931030273 	  Acc:17.866666793823242 	 Val_loss:1.8096681833267212 	 Val_acc:34.34000015258789


Epoch:1 	 Loss:1.6695348024368286 	  Acc:40.608333587646484 	 Val_loss:1.1889042854309082 	 Val_acc:60.630001068115234


Epoch:2 	 Loss:1.2952954769134521 	  Acc:53.584999084472656 	 Val_loss:0.7855485677719116 	 Val_acc:72.94999694824219


Epoch:3 	 Loss:0.9705101251602173 	  Acc:66.23332977294922 	 Val_loss:0.5505603551864624 	 Val_acc:83.0


...

Epoch:41 	 Loss:0.8540305495262146 	  Acc:70.80332946777344 	 Val_loss:0.46014994382858276 	 Val_acc:85.6199951171875


Epoch:42 	 Loss:0.8055254220962524 	  Acc:72.54666900634766 	 Val_loss:0.4349783957004547 	 Val_acc:87.20999908447266


Epoch:43 	 Loss:0.7757413983345032 	  Acc:73.77999877929688 	 Val_loss:0.4117286205291748 	 Val_acc:87.72000122070312


Epoch:44 	 Loss:0.7772415280342102 	  Acc:73.59832763671875 	 Val_loss:0.37575027346611023 	 Val_acc:88.52999877929688


Epoch:45 	 Loss:0.7433900237083435 	  Acc:74.61500549316406 	 Val_loss:0.3608364462852478 	 Val_acc:89.01000213623047


In [0]:
# counter = 0
# for i in train_data:
#   print(i)
#   print(i[0].numpy().shape)
#   print(i[1].numpy().shape)

#   counter += 1
#   if counter == 1:
#     break

# TF - 1.X

In [0]:
# in_a = tf.placeholder(dtype=tf.float32, shape=(2))
# in_b = tf.placeholder(dtype=tf.float32, shape=(2))
 
# def forward(x):
#   with tf.variable_scope("matmul", reuse=tf.AUTO_REUSE):

#     W = tf.get_variable("W", initializer=tf.ones(shape=(2,2)),
#                         regularizer=tf.contrib.layers.l2_regularizer(0.04))
#     b = tf.get_variable("b", initializer=tf.zeros(shape=(2)))
#     return W * x + b
 
# out_a = forward(in_a)
# out_b = forward(in_b)
 
# reg_loss = tf.losses.get_regularization_loss(scope="matmul")
 
# with tf.Session() as sess:
#   sess.run(tf.global_variables_initializer())
#   outs = sess.run([out_a, out_b, reg_loss],
#                   feed_dict={in_a: [1, 0], in_b: [0, 1]})

# TF - 2.X

In [0]:
# W = tf.Variable(tf.ones(shape=(2,2)), name="W")
# b = tf.Variable(tf.zeros(shape=(2)), name="b")

# @tf.function
# def forward(x):
#   return W * x + b

# out_a = forward([1,0])
# print(out_a)

## Create the small custom layer with @tf.function 

In [0]:
# tf.keras.backend.clear_session()

# # model archtecture 
# input_ = tf.keras.layers.Input(shape=(28,28,1))
# x = tf.keras.layers.Conv2D(128,
#                            (3,3),
#                            padding='same',
#                            activation='relu',
#                           #  kernel_initializer='he_normal',
#                            kernel_regularizer=tf.keras.regularizers.l2(l=0.02),
#                            kernel_constraint=tf.keras.constraints.MinMaxNorm())(input_)
# x = tf.keras.layers.LayerNormalization()(x)
# x = tf.keras.layers.BatchNormalization()(x)
# x = tf.keras.layers.MaxPool2D()(x)
# x = tf.keras.layers.Dropout(0.4, trainable=True)(x)
# for i in range(10):
#   # highway block
#   h = tf.keras.layers.Conv2D(128,
#                              (3,3),
#                              padding='same',
#                              activation='relu',
#                             #  kernel_initializer='he_normal',
#                              kernel_regularizer=tf.keras.regularizers.l2(l=0.02),
#                              kernel_constraint=tf.keras.constraints.MinMaxNorm())(x)
#   c = tf.keras.layers.Conv2D(128, 
#                              (3,3), 
#                              padding='same',
#                              activation='sigmoid',
#                              kernel_regularizer=tf.keras.regularizers.l2(l=0.02),
#                              kernel_constraint=tf.keras.constraints.MinMaxNorm(),
#                              bias_initializer = tf.keras.initializers.Constant(-3.))(x)
#   hc = tf.keras.layers.Multiply()([h,c])
#   ad = tf.keras.layers.Lambda(lambda x: 1.0 - x)(c)

#   xc = tf.keras.layers.Multiply()([x, ad])
#   x = tf.keras.layers.Add()([hc, xc])  
#   if i == 7 :
#     x = tf.keras.layers.MaxPool2D()(x)
#   x = tf.keras.layers.Dropout(0.4, trainable=True)(x)
# x = tf.keras.layers.GlobalAvgPool2D()(x)
# x = tf.keras.layers.LayerNormalization()(x)
# x = tf.keras.layers.BatchNormalization()(x)
# x = tf.keras.layers.Dropout(0.4, trainable=True)(x)
# x = tf.keras.layers.Dense(128, activation='relu')(x)
# x = tf.keras.layers.LayerNormalization()(x)
# x = tf.keras.layers.BatchNormalization()(x)
# x = tf.keras.layers.Dropout(0.4, trainable=True)(x)
# out_ = tf.keras.layers.Dense(10, activation='softmax')(x)

In [0]:
# Function hw model

def hw_model(x):
  h = tf.keras.layers.Conv2D(128,
                              (3,3),
                              padding='same',
                              activation='relu',
                              # kernel_initializer='he_normal',
                              # kernel_regularizer=tf.keras.regularizers.l2(l=0.02),
                              # kernel_constraint=tf.keras.constraints.MinMaxNorm()
                              )(x)

  c = tf.keras.layers.Conv2D(128, 
                              (3,3), 
                              padding='same',
                              activation='sigmoid',
                            #  kernel_regularizer=tf.keras.regularizers.l2(l=0.02),
                            #  kernel_constraint=tf.keras.constraints.MinMaxNorm(),
                              bias_initializer = tf.keras.initializers.Constant(-3.))(x)

  hc = tf.keras.layers.Multiply()([h,c])
  ad = tf.keras.layers.Lambda(lambda x: 1.0 - x)(c)

  xc = tf.keras.layers.Multiply()([x, ad])
  x = tf.keras.layers.Add()([hc, xc])
  return x

In [0]:
class hwnet(tf.keras.layers.Layer):
  def __init__(self, units, **kwargs):
    super(hwnet, self).__init__(**kwargs)
    self.units = units

    self.h = tf.keras.layers.Conv2D(self.units, 3, padding='same')
    self.c = tf.keras.layers.Conv2D(self.units, 3, padding='same')  
  # @tf.function
  def call(self, x, traininig=True):
    h_x = self.h(x)
    h_x = tf.nn.relu(h_x)

    c_x = self.c(x)
    c_x = tf.nn.sigmoid(c_x)
    
    hc = c_x * h_x + x*(1. - c_x)
    
    return hc

In [0]:
my_layer = hwnet(128)

In [0]:
input_ = tf.keras.layers.Input(shape=(28,28,1))
x = my_layer(input_)

mode = tf.keras.models.Model(input_, x)

In [70]:
mode.summary()

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        [(None, 28, 28, 1)]       0         
_________________________________________________________________
hwnet_8 (hwnet)              (None, 28, 28, 128)       2560      
Total params: 2,560
Trainable params: 2,560
Non-trainable params: 0
_________________________________________________________________


In [0]:
input_ = tf.keras.layers.Input(shape=(28,28,1))
x = hw_model(input_)
mode2 = tf.keras.models.Model(input_, x)

In [65]:
mode2.summary()

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_17 (Conv2D)              (None, 28, 28, 128)  1280        input_12[0][0]                   
__________________________________________________________________________________________________
conv2d_16 (Conv2D)              (None, 28, 28, 128)  1280        input_12[0][0]                   
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 28, 28, 128)  0           conv2d_17[0][0]                  
____________________________________________________________________________________________

In [0]:
m1 = mode.trainable_variables

In [0]:
m2 = mode.trainable_variables

# Funtion Tester

In [0]:
# Function hw model

def hw_model(x):
  h = tf.keras.layers.Conv2D(128,
                              (3,3),
                              padding='same',
                              activation='relu',
                              # kernel_initializer='he_normal',
                              # kernel_regularizer=tf.keras.regularizers.l2(l=0.02),
                              # kernel_constraint=tf.keras.constraints.MinMaxNorm()
                              )(x)

  c = tf.keras.layers.Conv2D(128, 
                              (3,3), 
                              padding='same',
                              activation='sigmoid',
                            #  kernel_regularizer=tf.keras.regularizers.l2(l=0.02),
                            #  kernel_constraint=tf.keras.constraints.MinMaxNorm(),
                              bias_initializer = tf.keras.initializers.Constant(-3.))(x)

  hc = tf.keras.layers.Multiply()([h,c])
  ad = tf.keras.layers.Lambda(lambda x: 1.0 - x)(c)

  xc = tf.keras.layers.Multiply()([x, ad])
  x = tf.keras.layers.Add()([hc, xc])
  return x

In [83]:
tf.keras.backend.clear_session()

# model archtecture 
input_ = tf.keras.layers.Input(shape=(28,28,1))
x = tf.keras.layers.Conv2D(128,
                           (3,3),
                           padding='same',
                           activation='relu',
                          #  kernel_initializer='he_normal',
                           kernel_regularizer=tf.keras.regularizers.l2(l=0.02),
                           kernel_constraint=tf.keras.constraints.MinMaxNorm())(input_)
x = tf.keras.layers.LayerNormalization()(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.Dropout(0.4, trainable=True)(x)
for i in range(10):
  # highway block
  x = hw_model(x)
  if i == 7 :
    x = tf.keras.layers.MaxPool2D()(x)
  x = tf.keras.layers.Dropout(0.4, trainable=True)(x)
x = tf.keras.layers.GlobalAvgPool2D()(x)
x = tf.keras.layers.LayerNormalization()(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.4, trainable=True)(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.LayerNormalization()(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.4, trainable=True)(x)
out_ = tf.keras.layers.Dense(10, activation='softmax')(x)


# define model
model = tf.keras.models.Model(input_, out_)
model.summary()

# plot model
tf.keras.utils.plot_model(model, show_shapes=True, dpi=80)

# define loss
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Set summary metrics
train_loss = tf.keras.metrics.Mean()
train_acc = tf.keras.metrics.SparseCategoricalAccuracy()

test_loss = tf.keras.metrics.Mean()
test_acc = tf.keras.metrics.SparseCategoricalAccuracy()


@tf.function
def train_step(img, label):
  with tf.GradientTape() as tape:
    pred = model(img, training=True)
    loss = loss_fn(label, pred)
  grad = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grad, model.trainable_variables))
  train_loss(loss)
  train_acc(label, pred)

@tf.function
def test_step(img, label):
  pred = model(img, training=False)
  loss = loss_fn(label, pred)
  test_loss(loss)
  test_acc(label, pred)


# Train Loop
n_epochs = 50
for epoch in range(n_epochs):
  train_loss.reset_states()
  train_acc.reset_states()
  test_loss.reset_states()
  test_acc.reset_states()

  for img, lab in tqdm_notebook(train_data):
    train_step(img, lab)

  for img, lab in test_data:
    test_step(img, lab)

  template = 'Epoch:{} \t Loss:{} \t  Acc:{} \t Val_loss:{} \t Val_acc:{}'
  print(template.format(epoch, train_loss.result(), train_acc.result()*100, test_loss.result(), test_acc.result()*100))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 28, 28, 128)  1280        input_1[0][0]                    
__________________________________________________________________________________________________
layer_normalization (LayerNorma (None, 28, 28, 128)  256         conv2d[0][0]                     
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 28, 28, 128)  512         layer_normalization[0][0]        
______________________________________________________________________________________________


Epoch:0 	 Loss:2.642965078353882 	  Acc:16.518333435058594 	 Val_loss:2.0929808616638184 	 Val_acc:19.68000030517578



Epoch:1 	 Loss:1.903296947479248 	  Acc:32.7066650390625 	 Val_loss:2.167893886566162 	 Val_acc:19.23000144958496



Epoch:2 	 Loss:1.5378406047821045 	  Acc:45.04833221435547 	 Val_loss:1.3899304866790771 	 Val_acc:47.71000289916992



Epoch:3 	 Loss:1.3074982166290283 	  Acc:53.393333435058594 	 Val_loss:1.2795355319976807 	 Val_acc:55.61000061035156


...


Epoch:45 	 Loss:0.5355451703071594 	  Acc:82.41500091552734 	 Val_loss:0.2166265994310379 	 Val_acc:93.5



Epoch:46 	 Loss:0.5657757520675659 	  Acc:81.55999755859375 	 Val_loss:0.2879483997821808 	 Val_acc:90.83999633789062



Epoch:47 	 Loss:0.5235001444816589 	  Acc:82.99833679199219 	 Val_loss:0.25443035364151 	 Val_acc:91.88999938964844



Epoch:48 	 Loss:0.5514200329780579 	  Acc:81.91666412353516 	 Val_loss:0.4504472315311432 	 Val_acc:85.25



Epoch:49 	 Loss:0.5988316535949707 	  Acc:80.40666198730469 	 Val_loss:0.24089853465557098 	 Val_acc:92.51000213623047


In [0]:
class hwnet(tf.keras.layers.Layer):
  def __init__(self, units, **kwargs):
    super(hwnet, self).__init__(**kwargs)
    self.units = units

    self.h = tf.keras.layers.Conv2D(self.units, 3, padding='same')
    self.c = tf.keras.layers.Conv2D(self.units, 3, padding='same', bias_initializer=tf.keras.initializers.Constant(-3.))  
  # @tf.function
  def call(self, x, traininig=True):
    h_x = self.h(x)
    h_x = tf.nn.relu(h_x)

    c_x = self.c(x)
    c_x = tf.nn.sigmoid(c_x)
    
    hc = c_x * h_x + x*(1. - c_x)
    
    return hc

In [86]:
tf.keras.backend.clear_session()

# model archtecture 
input_ = tf.keras.layers.Input(shape=(28,28,1))
x = tf.keras.layers.Conv2D(128,
                           (3,3),
                           padding='same',
                           activation='relu',
                          #  kernel_initializer='he_normal',
                           kernel_regularizer=tf.keras.regularizers.l2(l=0.02),
                           kernel_constraint=tf.keras.constraints.MinMaxNorm())(input_)
x = tf.keras.layers.LayerNormalization()(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.Dropout(0.4, trainable=True)(x)
for i in range(10):
  # highway block
  x = hwnet(128)(x)
  if i == 7 :
    x = tf.keras.layers.MaxPool2D()(x)
  x = tf.keras.layers.Dropout(0.4, trainable=True)(x)
x = tf.keras.layers.GlobalAvgPool2D()(x)
x = tf.keras.layers.LayerNormalization()(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.4, trainable=True)(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.LayerNormalization()(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.4, trainable=True)(x)
out_ = tf.keras.layers.Dense(10, activation='softmax')(x)


# define model
model = tf.keras.models.Model(input_, out_)
model.summary()

# plot model
tf.keras.utils.plot_model(model, show_shapes=True, dpi=80)

# define loss
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Set summary metrics
train_loss = tf.keras.metrics.Mean()
train_acc = tf.keras.metrics.SparseCategoricalAccuracy()

test_loss = tf.keras.metrics.Mean()
test_acc = tf.keras.metrics.SparseCategoricalAccuracy()


@tf.function
def train_step(img, label):
  with tf.GradientTape() as tape:
    pred = model(img, training=True)
    loss = loss_fn(label, pred)
  grad = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grad, model.trainable_variables))
  train_loss(loss)
  train_acc(label, pred)

@tf.function
def test_step(img, label):
  pred = model(img, training=False)
  loss = loss_fn(label, pred)
  test_loss(loss)
  test_acc(label, pred)


# Train Loop
n_epochs = 50
for epoch in range(n_epochs):
  train_loss.reset_states()
  train_acc.reset_states()
  test_loss.reset_states()
  test_acc.reset_states()

  for img, lab in tqdm_notebook(train_data):
    train_step(img, lab)

  for img, lab in test_data:
    test_step(img, lab)

  template = 'Epoch:{} \t Loss:{} \t  Acc:{} \t Val_loss:{} \t Val_acc:{}'
  print(template.format(epoch, train_loss.result(), train_acc.result()*100, test_loss.result(), test_acc.result()*100))

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 128)       1280      
_________________________________________________________________
layer_normalization (LayerNo (None, 28, 28, 128)       256       
_________________________________________________________________
batch_normalization (BatchNo (None, 28, 28, 128)       512       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 128)       0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 128)       0         
_________________________________________________________________
hwnet (hwnet)                (None, 14, 14, 128)       295168


Epoch:0 	 Loss:2.6473326683044434 	  Acc:17.434999465942383 	 Val_loss:2.3466312885284424 	 Val_acc:9.809999465942383



Epoch:1 	 Loss:1.8086100816726685 	  Acc:36.46666717529297 	 Val_loss:1.9726921319961548 	 Val_acc:23.3700008392334



Epoch:2 	 Loss:1.4664957523345947 	  Acc:47.71666717529297 	 Val_loss:1.1578290462493896 	 Val_acc:57.06999969482422



Epoch:3 	 Loss:1.2535818815231323 	  Acc:55.323333740234375 	 Val_loss:0.8606882095336914 	 Val_acc:71.81999969482422


...


Epoch:45 	 Loss:0.6148582696914673 	  Acc:79.45999908447266 	 Val_loss:0.2896623909473419 	 Val_acc:91.00999450683594



Epoch:46 	 Loss:0.5815957188606262 	  Acc:80.47000122070312 	 Val_loss:0.3121838867664337 	 Val_acc:89.80000305175781



Epoch:47 	 Loss:0.5675150156021118 	  Acc:81.28166961669922 	 Val_loss:0.2772147059440613 	 Val_acc:91.1199951171875



Epoch:48 	 Loss:0.6114705801010132 	  Acc:79.69667053222656 	 Val_loss:0.28582069277763367 	 Val_acc:90.93000030517578



Epoch:49 	 Loss:0.5816935896873474 	  Acc:80.68666076660156 	 Val_loss:0.2678208351135254 	 Val_acc:91.75
